# Variational Autoencoder met Fashion MNIST

Een Variational Autoencoder (VAE) is een type autoencoder dat wordt gebruikt voor het genereren van nieuwe voorbeelden die vergelijkbaar zijn met de dataset waarop het is getraind. 
Het bestaat uit een encoder die de input data naar een lagere-dimensionale latente ruimte projecteert, en een decoder die uit deze latente ruimte nieuwe data reconstrueert.

Overzicht:
* Importeren van bibliotheken en dataset
* Definiëren van de VAE-architectuur
* Trainen van het VAE-model
* Genereren van nieuwe afbeeldingen met de VAE
## Importeren van packages en dataset

Eerst importeren we alle benodigde Python-bibliotheken voor het bouwen, trainen en visualiseren van onze VAE.
We gebruiken Pytorch voor het bouwen van het neurale netwerk, matplotlib voor visualisaties en NumPy voor numerieke berekeningen.
Daarna laden we de Fashion MNIST dataset, normaliseren de pixelwaarden naar de range [0,1] 
en splitsen de dataset in een trainings- en testset. We gebruiken DataLoader om mini-batches te maken voor training.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# Check of GPU beschikbaar is
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Apparaat gebruikt voor training: {device}")

# Definieer transformatie (normaliseren van beelden)
transform = transforms.Compose([
    transforms.ToTensor(),  # Converteer afbeeldingen naar PyTorch tensors
])

# Laad de Fashion MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

# Creëer DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"Aantal trainingsbatches: {len(train_loader)}, Aantal testbatches: {len(test_loader)}")

## Definiëren van de VAE-architectuur

De architectuur van onze Variational Autoencoder bestaat uit een encoder die de inputbeelden naar een lagere-dimensionale latente ruimte projecteert en een decoder die deze latente ruimte gebruikt om de afbeeldingen opnieuw te genereren.
De verliesfunctie voor onze VAE, bestaande uit de som van de reconstructieverlies (binary cross-entropy)
en de KL-divergentie. We gebruiken de Adam optimizer voor het bijwerken van de gewichten van het netwerk.

In [ ]:
# dit gaat niet letterlijk als oefening gegeven worden in het praktijk gedeelte
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dim):
        super(VariationalAutoencoder, self).__init__()
        
        self.latent_dim = latent_dim

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1), # aantal in kanalen is 1 want grijswaardenbeelden
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # 64 * 7 * 7
            nn.ReLU(),

            nn.Flatten()
        )

        # bottleneck
        # mean
        self.fc_mu  = nn.Linear(64*7*7, latent_dim)
        # std
        self.fc_logvar = nn.Linear(64*7*7, latent_dim)

        self.decoder_input = nn.Linear(latent_dim, 64*7*7)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding= 1, output_padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        x = self.encoder(x)
        return self.fc_mu(x), self.fc_logvar(x)

    def decode(self, x):
        x = self.decoder_input(x)
        # resize van 64*7*7 -> (64, 7, 7)
        x = x.view(-1, 64, 7, 7)
        return self.decoder(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + std * eps

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
latent_dim = 2
model = VariationalAutoencoder(latent_dim).to(device)
print(model)

optimizer = optim.Adam(model.parameters(), lr = 0.001)
# functie moet je niet zelf kunnen opstellen
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')

    kl_loss = -0.5 * torch.sum(1+logvar - mu.pow(2) - logvar.exp())

    return recon_loss + kl_loss

## Trainen van het VAE model

In deze cel trainen we het VAE-model met de trainingsgegevens. 
Voor elke epoch voeren we een forward pass, berekenen we het verlies, en voeren we een backward pass uit om de gewichten bij te werken.

In [ ]:
num_epochs = 10

model.train()
for epoch in range(num_epochs):
    running_loss = 0
    for images, _ in train_loader:
        images = images.to(device)

        optimizer.zero_grad()
        images2, mu, logvar = model(images)
        loss = vae_loss(images2, images, mu, logvar)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch}/{num_epochs}, loss: {running_loss/len(train_loader)}")

## Genereren van nieuwe resultaten

Eerst bestuderen we de latente ruimte door de testset in kaart te brengen in de 2D latente ruimte van de VAE. Hiermee kunnen we zien hoe goed de VAE leert om vergelijkbare afbeeldingen bij elkaar te plaatsen.
Daarna genereren we nieuwe afbeeldingen door willekeurige punten te nemen uit de latente ruimte 
en deze te decoderen met behulp van de decoder van de VAE.

In [ ]:
# Genereer nieuwe afbeeldingen van random punten in de latente ruimte
n = 10  # Aantal afbeeldingen per rij/kolom
figure = np.zeros((28 * n, 28 * n))

# Uniform verdeeld random punten in de latente ruimte
grid_x = np.linspace(-3, 3, n)
grid_y = np.linspace(-3, 3, n)

model.eval()
with torch.no_grad():
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = torch.tensor([[xi, yi]], dtype=torch.float32).to(device)
            x_decoded = model.decode(z_sample).cpu().numpy()
            digit = x_decoded[0].reshape(28, 28)
            figure[i * 28: (i + 1) * 28, j * 28: (j + 1) * 28] = digit

plt.figure(figsize=(8, 8))
plt.imshow(figure, cmap='Greys_r')
plt.title("Gegenereerde afbeeldingen van random punten in de latente ruimte")

# Oefening: denoising autoencoder

Een denoising autoencoder (DAE) is een type autoencoder dat is ontworpen om ruis uit de invoergegevens te verwijderen. Het is een populaire techniek in deep learning voor het voorverwerken van gegevens, het leren van robuuste representaties, en voor compressiedoeleinden.

In deze oefening ga je een denoising autoencoder implementeren in PyTorch. Deze oefening omvat de volgende stappen:

* Data voorbereiden: We gebruiken de Fashion MNIST dataset, voegen ruis toe aan de afbeeldingen en schalen de gegevens naar het bereik [0, 1]. (Dit deel krijg je hieronder)
* Model bouwen: We bouwen een eenvoudig denoising autoencoder model met een encoder en een decoder.
* Model trainen: We trainen het model met ruisachtige afbeeldingen als invoer en niet-vervuilde afbeeldingen als doel.
* Resultaten evalueren: We testen het model door enkele ruisachtige afbeeldingen door het netwerk te laten gaan en hun gereconstrueerde versies weer te geven.

In [ ]:
# Data voorbereiden

# Importeren van benodigde bibliotheken
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Controleer of er een GPU beschikbaar is, zo niet gebruik de CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data-transformatie: normaliseer de afbeeldingen zodat de pixelwaarden tussen 0 en 1 liggen
transform = transforms.Compose([
    transforms.ToTensor()  # Converteert beeld naar tensor en schaalt automatisch naar [0, 1]
])

# FashionMNIST dataset downloaden en laden
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

# DataLoader voor batches van de trainings- en testdata
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

print("Data geladen en DataLoader klaar.")

# Functie om ruis aan de afbeeldingen toe te voegen
def add_noise(imgs, noise_factor=0.5):
    noisy_imgs = imgs + noise_factor * torch.randn_like(imgs)
    noisy_imgs = torch.clamp(noisy_imgs, 0., 1.)
    return noisy_imgs

# Visualisatie van enkele ruisachtige afbeeldingen
examples = enumerate(test_loader)
batch_idx, (example_data, _) = next(examples)

noisy_imgs = add_noise(example_data)

fig, axes = plt.subplots(1, 5, figsize=(15, 4))
for i in range(5):
    axes[i].imshow(noisy_imgs[i].squeeze().numpy(), cmap='gray')
    axes[i].axis('off')
plt.show()

In [ ]:
# Model bouwen
class DenoisingAutoencoder(nn.Module):
    def __init__(self, ):
        super(DenoisingAutoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1), # aantal in kanalen is 1 want grijswaardenbeelden
            nn.ReLU(),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1), # 32 * 7 * 7
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=7, stride=1, padding=0) # 64 * 1 * 1
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding= 1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding= 1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x  = self.encoder(x)
        return self.decoder(x)

model = DenoisingAutoencoder().to(device)

In [ ]:
# Model trainen

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(num_epochs):
    running_loss = 0
    for images, _ in train_loader:
        images = images.to(device)
        images_noise = add_noise(images).to(device)
        
        optimizer.zero_grad()
        images_de = model(images_noise)
        loss = criterion(images_de, images)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch}/{num_epochs}, loss: {running_loss/len(train_loader)}")

In [ ]:
# Resultaten visualiseren

model.eval()
test_examples = None
with torch.no_grad():
    for batch in test_loader:
        imgs, _ = batch
        noisy_imgs = add_noise(imgs).to(device)
        outputs = model(noisy_imgs)
        test_examples = (imgs, noisy_imgs, outputs)
        break

# Plot de resultaten
orig, noisy_imgs, outputs = test_examples
fig, axes = plt.subplots(3, 5, figsize=(15, 6))
for i in range(5):
    # Originele afbeeldingen
    print(orig[i].shape)
    axes[0, i].imshow(orig[i].cpu().squeeze().numpy(), cmap='gray')
    axes[0, i].axis('off')

    # Ruisachtige afbeeldingen
    axes[1, i].imshow(noisy_imgs[i].cpu().squeeze().numpy(), cmap='gray')
    axes[1, i].axis('off')
    
    # Gereconstrueerde afbeeldingen
    axes[2, i].imshow(outputs[i].cpu().squeeze().numpy(), cmap='gray')
    axes[2, i].axis('off')
plt.show()